In [106]:
import numpy as np
from math import inf
from utilities import *
from time import perf_counter
from sortedcollections import SortedDict
from disjoint_set import DisjointSet
from matheuristic import *
from copy import deepcopy

In [107]:

name, capacity, node_data = read_instance("../instances/c207.txt")
ins = instance(name, capacity, node_data, 100)
global xcoords, ycoords, latest, earliest, D, Q
xcoords, ycoords = ins.xcoords, ins.ycoords
earliest, latest = ins.earliest, ins.latest
D = ins.cost
Q = ins.capacity
s, cost = prim(ins, vis = False, initial = True)

In [108]:
def copy(s):
    return (s[0].copy(), s[1].copy(), s[2].copy(), s[3].copy())

In [109]:
def random_father(s): 
    parent = s[0] 
    gate = s[1] 
    load = s[2] 
    arrival = s[3] 

    n = len(parent) 
    j = np.random.randint(1,n) 
    # print(j, parent[j])
    parent[j] = -1 

    ds = get_disjoint(parent)

    sets = list(ds.itersets())
    con, disc = sets[0], sets[1]

    l = len(disc)
    load[gate[j]] -= l 

    for i in sorted(con, key = lambda x: random()):
        if load[gate[i]] + l <= Q: 
            
            parent[j] = i
            subroot = gate[i] if i != 0 else j
            for jj in sorted(disc, key = lambda x: arrival[x]):
                gate[jj] = subroot
                arrival[jj] = max(arrival[parent[jj]] + distance(jj, parent[jj]), earliest[jj]) # hacer esto por orde de llegada, asi se me hace mas facil
            load[gate[j]] += l
            return (parent, gate, load, arrival)

In [110]:
%%timeit
x = branch_to_branch(copy(s))

NameError: name 'Q' is not defined

In [71]:
%%timeit
x = random_father(copy(s))

566 µs ± 4.27 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [211]:
class branch_bound:
    def __init__(self, s):
        self.nodes = s
        self.best_solution = None
        self.best_cost = inf

        parent = SortedDict()
        arrival = SortedDict()
        gate = SortedDict()
        load = SortedDict()

        parent[0] = -1
        arrival[0] = gate[0] = load[0] = 0
        for i in self.nodes:
            arrival[i] = gate[i] = load[i] = 0

        nodes_left = set(self.nodes)
        
        s = (parent, gate, load, arrival)
        self.explore(s, 0, nodes_left)

    def explore(self, s, cost, nodes_left):
        if len(nodes_left) == 0:
            if cost < self.best_cost:
                self.best_cost = cost
                self.best_solution = deepcopy(s)

        else:
            for j in nodes_left:
                for i in s[0].keys():
                    if cost + distance(i, j) < self.best_cost:
                        parent, gate, load, arrival = deepcopy(s)
                        parent[j] = i
                        if i == 0:
                            gate[j] = j
                        else:
                            gate[j] = gate[i]
    
                        if arrival[i] + distance(i,j) <= latest[j] and load[gate[j]] < Q: # isFeasible() # reescribir
                            
                            load[gate[j]] += 1
                            arrival[j] = arrival[i] + distance(i,j)
                            if arrival[j] < earliest[j]:
                                arrival[j] = earliest[j]
                            
                            self.explore((parent, gate, load, arrival), cost + distance(i, j), nodes_left - {j})
                            load[gate[j]] -= 1

In [212]:
%%timeit
sol = branch_bound([1,2,4,5])
bb = sol.best_solution

15.4 ms ± 366 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [213]:
class branch_bound:
    def __init__(self, branch):
        self.nodes = [0] + branch
        self.best_solution = None
        self.best_cost = inf
        self.t = {num:j for j,num in enumerate(self.nodes)}
        self.decode = {j:num for j,num in enumerate(self.nodes)}
        self.decode[-1] = -1
        self.n = n = len(self.nodes)

        parent = np.zeros(n, dtype = int)
        gate = np.zeros(n, dtype = int)
        load = np.zeros(n, dtype = int)
        arrival = np.zeros(n)
        parent[0] = -1

        nodes_left = set(self.nodes[1:])
        itree = set()
        itree.add(0)
        
        s = (parent, gate, load, arrival)
        self.explore(s, 0, nodes_left, itree)

    def explore(self, s, cost, nodes_left, itree):
        if len(nodes_left) == 0:
            if cost < self.best_cost:
                self.best_cost = cost
                self.best_solution = deepcopy(s)
        else:
            for j in nodes_left:
                kj = self.t[j]
                for i in itree:
                    ki = self.t[i]
                    
                    if cost + distance(i, j) < self.best_cost:
                        
                        parent, gate, load, arrival = deepcopy(s)
                        parent[kj] = ki
                        if i == 0:
                            gate[kj] = kj
                        else:
                            gate[kj] = gate[ki]
    
                        if arrival[ki] + distance(i,j) <= latest[j] and load[gate[kj]] < Q: # isFeasible() # reescribir
                            
                            load[gate[kj]] += 1
                            arrival[kj] = arrival[ki] + distance(i,j)
                            if arrival[kj] < earliest[j]:
                                arrival[kj] = earliest[j]
                            
                            self.explore((parent, gate, load, arrival), cost + distance(i, j), nodes_left - {j}, itree | {j})
                            load[gate[kj]] -= 1
    def give_solution(self):
        parent = SortedDict()
        arrival = SortedDict()
        gate = SortedDict()
        load = SortedDict()
        for ki in range(self.n):
            i = self.decode[ki]
            parent[i] = self.decode[self.best_solution[0][ki]]
            gate[i] = self.decode[self.best_solution[1][ki]]
            load[i] = self.best_solution[2][ki]
            arrival[i] = self.best_solution[3][ki]
        return(parent, gate, load, arrival)

In [214]:
%%timeit
sol = branch_bound([1,2,4,5])
bb = sol.give_solution()

2.23 ms ± 42.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [168]:
print(sol.best_solution)

(array([-1,  1,  2,  3,  4]), array([0, 0, 0, 0, 0]), array([4, 0, 0, 0, 0]), array([   0.,  302.,  157., 1247.,   15.]))


In [ ]:
class tree:
    def __init__(self, n):
        self.parent = np.ones(n, dtype = int) * -1
        self.gate = np.zeros(n, dtype = int)
        self.load = np.zeros(n, dtype = int)
        self.arrival = np.zeros(n)

    def connect(self, k, j):
        """k: parent, j: children"""
        self.gate[j] = gate = self.gate[k]
        self.load[gate] += 1
        self.parent[j] = k
        self.arrival[j] = distance(k,j)
        if not  self.arrival[j] >= earliest[j]:
            self.arrival[j] = earliest[j]

    def connect_to_root(self, j):
        """k: parent, j: children"""
        self.gate[j] = j
        self.load[self.gate[j]] += 1
        self.parent[j] = 0
        self.arrival[j] = distance(0,j)
        if not  self.arrival[j] >= earliest[j]:
            self.arrival[j] = earliest[j]

    def fitness(self):
        cost = 0 
        feasible = True
        for j,k in enumerate(self.parent):
            if j != 0:
                cost += distance(k,j)
                arr = self.arrival[j]
                lat = latest[j]
                if  arr > lat:
                    feasible = False
                    cost += (arr - lat) * PENALIZATION

        return cost, feasible

In [ ]:
class branch_bound:
    def __init__(self, branch):
        self.nodes = [0] + branch
        self.best_solution = None
        self.best_cost = inf
        self.t = {num:j for j,num in enumerate(self.nodes)}
        self.decode = {j:num for j,num in enumerate(self.nodes)}
        self.decode[-1] = -1
        self.n = n = len(self.nodes)

        parent = np.zeros(n, dtype = int)
        gate = np.zeros(n, dtype = int)
        load = np.zeros(n, dtype = int)
        arrival = np.zeros(n)
        parent[0] = -1

        nodes_left = set(self.nodes[1:])
        itree = set()
        itree.add(0)
        
        s = (parent, gate, load, arrival)
        self.explore(s, 0, nodes_left, itree)

    def explore(self, s, cost, nodes_left, itree):
        if len(nodes_left) == 0:
            if cost < self.best_cost:
                self.best_cost = cost
                self.best_solution = deepcopy(s)
        else:
            for j in nodes_left:
                kj = self.t[j]
                for i in itree:
                    ki = self.t[i]
                    
                    if cost + distance(i, j) < self.best_cost:
                        
                        parent, gate, load, arrival = deepcopy(s)
                        parent[kj] = ki
                        if i == 0:
                            gate[kj] = kj
                        else:
                            gate[kj] = gate[ki]
    
                        if arrival[ki] + distance(i,j) <= latest[j] and load[gate[kj]] < Q: # isFeasible() # reescribir
                            
                            load[gate[kj]] += 1
                            arrival[kj] = arrival[ki] + distance(i,j)
                            if arrival[kj] < earliest[j]:
                                arrival[kj] = earliest[j]
                            
                            self.explore((parent, gate, load, arrival), cost + distance(i, j), nodes_left - {j}, itree | {j})
                            load[gate[kj]] -= 1
    def give_solution(self):
        parent = SortedDict()
        arrival = SortedDict()
        gate = SortedDict()
        load = SortedDict()
        for ki in range(self.n):
            i = self.decode[ki]
            parent[i] = self.decode[self.best_solution[0][ki]]
            gate[i] = self.decode[self.best_solution[1][ki]]
            load[i] = self.best_solution[2][ki]
            arrival[i] = self.best_solution[3][ki]
        return(parent, gate, load, arrival)